In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/home/ueyama/workspace/ClimaX/src")

In [8]:
import os

from src.climax.arch import ClimaX
from src.climax.my_arch3 import ClimaX3
from src.climax.my_arch2 import ClimaX2
from climax.utils.metrics import (
    lat_weighted_mse,
)

import torch
from torchviz import make_dot
import numpy as np

#### Config

In [5]:
default_vars = [
          "land_sea_mask",
          "orography",
          "lattitude",
          "2m_temperature",
          "10m_u_component_of_wind",
          "10m_v_component_of_wind",
          "geopotential_50",
          "geopotential_250",
          "geopotential_500",
          "geopotential_600",
          "geopotential_700",
          "geopotential_850",
          "geopotential_925",
          "u_component_of_wind_50",
          "u_component_of_wind_250",
          "u_component_of_wind_500",
          "u_component_of_wind_600",
          "u_component_of_wind_700",
          "u_component_of_wind_850",
          "u_component_of_wind_925",
          "v_component_of_wind_50",
          "v_component_of_wind_250",
          "v_component_of_wind_500",
          "v_component_of_wind_600",
          "v_component_of_wind_700",
          "v_component_of_wind_850",
          "v_component_of_wind_925",
          "temperature_50",
          "temperature_250",
          "temperature_500",
          "temperature_600",
          "temperature_700",
          "temperature_850",
          "temperature_925",
          "relative_humidity_50",
          "relative_humidity_250",
          "relative_humidity_500",
          "relative_humidity_600",
          "relative_humidity_700",
          "relative_humidity_850",
          "relative_humidity_925",
          "specific_humidity_50",
          "specific_humidity_250",
          "specific_humidity_500",
          "specific_humidity_600",
          "specific_humidity_700",
          "specific_humidity_850",
          "specific_humidity_925",
        ]

In [9]:
config1 =   {"default_vars": default_vars,
            "img_size":[32, 64],
            "patch_size":2,
            "embed_dim":1024,
            "depth":8,
            "decoder_depth":2,
            "num_heads":16,
            "mlp_ratio":4.0,
            "drop_path":0.1,
            "drop_rate":0.1,
            "parallel_patch_embed":True}

In [10]:
config2 =   {"default_vars": default_vars,
            "img_size":[32, 64],
            "patch_size":2,
            "embed_dim":1024,
            "depth":8,
            "decoder_depth":2,
            "num_heads":16,
            "num_representative":2,
            "mlp_ratio":4.0,
            "drop_path":0.1,
            "drop_rate":0.1,
            "parallel_patch_embed":True}

In [6]:
config3 =   {"default_vars": default_vars,
            "img_size":[32, 64],
            "patch_size":2,
            "embed_dim":1024,
            "depth":8,
            "decoder_depth":2,
            "num_heads":16,
            "num_representative":2,
            "mlp_ratio":4.0,
            "drop_path":0.1,
            "drop_rate":0.1,
            "parallel_patch_embed":True}

#### パラメータ数

ClimaX

In [12]:
climax = ClimaX(**config1).cuda()
x = sum([param.numel() for param in climax.parameters()])
print(x)

108088512


ClimaX2

In [13]:
for i in [1, 2, 4, 8]:
    config2["num_representative"] = i
    my_climax2 = ClimaX2(**config2).cuda()
    x = sum([param.numel() for param in my_climax2.parameters()])
    print(x)

196262080
76554112
33215488
15675520


ClimaX3

In [9]:
for i in [1, 2, 4, 8]:
    config3["num_representative"] = i
    my_climax3 = ClimaX3(**config3).cuda()
    x = sum([param.numel() for param in my_climax3.parameters()])
    print(x)

196276416
59746176
20579328
8270976


#### 推論

準備

In [15]:
lat = np.load(os.path.join("/home/ueyama/workspace/ClimaX/dataset/5.625deg_npz_06_18", "lat.npy"))
x = torch.randn(8, 48, 32, 64).cuda()
y = torch.randn(8, 3, 32, 64).cuda()
lead_times = torch.randn(1).cuda()
variables = default_vars
out_variables = ["temperature_50", "temperature_250", "temperature_500"]
metric = None
lat = None

ClimaX

In [21]:
climax = ClimaX(**config1).cuda()

In [22]:
%%timeit
x = torch.randn(8, 48, 32, 64).cuda()
y = torch.randn(8, 3, 32, 64).cuda()
_ =  climax(x, y, lead_times, variables, out_variables, metric, lat)

142 ms ± 383 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


ClimaX2

In [14]:
config2["num_representative"] = 2
my_climax2 = ClimaX2(**config2).cuda()

In [15]:
%%timeit
x = torch.randn(8, 48, 32, 64).cuda()
y = torch.randn(8, 3, 32, 64).cuda()
_ = my_climax2(x, y, lead_times, variables, out_variables, metric, lat)

340 ms ± 94.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


ClimaX3

In [24]:
config3["num_representative"] = 2
my_climax3 = ClimaX3(**config3).cuda()

In [25]:
%%timeit
x = torch.randn(8, 48, 32, 64).cuda()
y = torch.randn(8, 3, 32, 64).cuda()
_ = my_climax3(x, y, lead_times, variables, out_variables, metric, lat)

335 ms ± 93.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


ClimaX3 regional

In [16]:
config3_region =   {"default_vars": default_vars,
                    "img_size":[8, 16],
                    "patch_size":2,
                    "embed_dim":1024,
                    "depth":8,
                    "decoder_depth":2,
                    "num_heads":16,
                    "num_representative":2,
                    "mlp_ratio":4.0,
                    "drop_path":0.1,
                    "drop_rate":0.1,
                    "parallel_patch_embed":True}

In [19]:
config3_region["num_representative"] = 2
my_climax3 = ClimaX3(**config3_region).cuda()

In [20]:
%%timeit
x = torch.randn(8, 48, 8, 16).cuda()
y = torch.randn(8, 3, 8, 16).cuda()
_ = my_climax3(x, y, lead_times, variables, out_variables, metric, lat)

12.9 ms ± 853 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### line profiler

In [29]:
lat = np.load(os.path.join("/home/ueyama/workspace/ClimaX/dataset/5.625deg_npz_06_18", "lat.npy"))
x = torch.randn(8, 48, 32, 64).cuda()
y = torch.randn(8, 3, 32, 64).cuda()
lead_times = torch.randn(1).cuda()
variables = default_vars
out_variables = ["temperature_50", "temperature_250", "temperature_500"]
metric = None
lat = None

In [26]:
%load_ext line_profiler

In [27]:
def hoge(x0, y, lead_times, variables, out_variables, metric, lat):
    # out_transformers = my_climax.forward_encoder(x, lead_times, variables=variables)  # R, B, L, D

    if isinstance(variables, list):
        variables = tuple(variables)
    # x0 = x
    xs = []
    for k in range(my_climax3.num_representatives):
        # tokenize each variable separately
        embeds = []
        var_ids = my_climax3.get_var_ids(variables, x0.device)

        if my_climax3.parallel_patch_embed:
            x = my_climax3.token_embeds(x0, var_ids)  # B, V, L, D
        else:
            for i in range(len(var_ids)):
                id = var_ids[i]
                embeds.append(my_climax3.token_embeds[k][id](x0[:, i : i + 1]))
            x = torch.stack(embeds, dim=1)  # B, V, L, D

        # add variable embedding
        var_embed = my_climax3.get_var_emb(my_climax3.var_embed[k], variables)
        x = x + var_embed.unsqueeze(2)  # B, V, L, D

        # variable aggregation
        # x = my_climax3.aggregate_variables(x, k)  # B, L, D
        b, _, l, _ = x.shape
        x = torch.einsum("bvld->blvd", x)
        x = x.flatten(0, 1)  # BxL, V, D

        # var_query = my_climax3.var_query[k].repeat_interleave(x.shape[0], dim=0) # BxL, 1, D
        var_query = my_climax3.var_query[k].repeat_interleave(x.shape[0], dim=0) # BxL, 1, D

        x, _ = my_climax3.var_agg[k](var_query, x, x)  # BxL, D
        x = x.squeeze()

        x = x.unflatten(dim=0, sizes=(b, l))  # B, L, D

        # add pos embedding
        x = x + my_climax3.pos_embed[k]

        # add lead time embedding
        lead_time_emb = my_climax3.lead_time_embed(lead_times.unsqueeze(-1))  # B, D
        lead_time_emb = lead_time_emb.unsqueeze(1)
        x = x + lead_time_emb  # B, L, D

        x = my_climax3.pos_drop(x)

        xs.append(x.unsqueeze(0)) # R, B, L, D

    xs = torch.cat(xs, dim=0) # R, B, L, D

    batch_size = xs.shape[1]
    # apply Transformer blocks

    for i in range(my_climax3.depth):
        xs = my_climax3.blocks[i](xs)
        xs = my_climax3.norm[i](xs)
        if i < my_climax3.depth - 1:
            xo = my_climax3.cross_over[i]
            xs = torch.einsum("rbld->blrd", xs) # R, B, L, D
            xs = torch.reshape(xs, (-1, *xs.shape[2:])) # (B, L), R, D
            xs = xo(xs) # (B, L), R, D
            xs = torch.reshape(xs, (batch_size, -1, *xs.shape[1:])) # B, L, R, D
            xs = torch.einsum("blrd->rbld", xs)

    preds = []
    for head, output in zip(my_climax3.heads, xs):
        preds.append(head(output))
    preds = sum(preds)/len(preds)  # B, L, V*p*p

    preds = my_climax3.unpatchify(preds)
    out_var_ids = my_climax3.get_var_ids(tuple(out_variables), preds.device)
    preds = preds[:, out_var_ids]

    if metric is None:
        loss = None
    else:
        loss = [m(preds, y, out_variables, lat) for m in metric]

    return loss, preds


In [28]:
%lprun -f hoge hoge(x, y, lead_times, variables, out_variables, metric, lat)

Timer unit: 1e-09 s

Total time: 0.133942 s
File: /tmp/ipykernel_1792597/1587645711.py
Function: hoge at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def hoge(x0, y, lead_times, variables, out_variables, metric, lat):
     2                                               # out_transformers = my_climax.forward_encoder(x, lead_times, variables=variables)  # R, B, L, D
     3                                           
     4         1       9168.0   9168.0      0.0      if isinstance(variables, list):
     5         1       3418.0   3418.0      0.0          variables = tuple(variables)
     6                                               # x0 = x
     7         1       1198.0   1198.0      0.0      xs = []
     8         3       6844.0   2281.3      0.0      for k in range(my_climax3.num_representatives):
     9                                                   # tokenize each variable separately
    10         2   

#### Class check

Linear3d

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint
from torch.jit import Final
from torch import Tensor
from torch.nn.parameter import Parameter
import math

class Linear3d(nn.Module):
    def __init__(self, input_dim: int, output_dim: int, extra_dim: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.extra_dim = extra_dim
        self.weight = Parameter(torch.empty((input_dim, output_dim, extra_dim)))
        if bias:
            self.bias = Parameter(torch.empty(output_dim, extra_dim))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        print(input.shape)
        print(self.weight.shape)
        print(self.bias.shape)
        input = torch.einsum("bncr, cdr->bndr",  (input, self.weight))
        if self.bias is not None:
            input = input + self.bias.view(1, *self.bias.shape)
        return input

    def extra_repr(self) -> str:
        return f'input_dim={self.input_dim}, output_dim={self.output_dim}, bias={self.bias is not None}'


In [26]:
linear = Linear3d(10,30,3)

In [27]:
x = torch.randn(4, 40, 10, 3) # B, N, C, R
x = linear(x)
x.shape

torch.Size([4, 40, 10, 3])
torch.Size([10, 30, 3])
torch.Size([30, 3])


torch.Size([4, 40, 30, 3])